In [ ]:
# Copied from https://www.kaggle.com/guimingjiang/1st-place-process-mel-spectrograms-256x256

In [ ]:
import numpy as np
import pandas as pd
import librosa
from collections import defaultdict

In [ ]:
train = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/train.csv')
test = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
# generate mel spectrogram

def mel_spectrogram(sensor):
    spec = librosa.feature.melspectrogram(y=sensor, sr=48000, n_mels=256, hop_length=60001//256+1)
    return librosa.power_to_db(spec)

In [ ]:
def generate_spectrograms_for_df(df, dataset):
    # Enumerate across all datasets
    for i, segment_id in enumerate(df['segment_id'].values):
        sensors = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/{dataset}/{segment_id}.csv', memory_map=True)
        sensors.fillna(0, inplace=True)
        
        signals = []
        for i in range(10):
            name = f'sensor_{i+1}'
            sensor = sensors[name].values
            signals.append(mel_spectrogram(sensor))
        
        # Cast to uint8 to save memory
        signals = np.array(signals, dtype='uint8')

        np.savez_compressed(f'{dataset}/{segment_id}-spec.npz', signals)

In [ ]:
!mkdir train test

In [ ]:
generate_spectrograms_for_df(train, 'train')
generate_spectrograms_for_df(test, 'test')